In [3]:
# Cell 1: Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
import time 
from tqdm import tqdm # For a nice progress bar

# Add the 'src' folder to the path
sys.path.insert(0, os.path.abspath('../src'))

# --- Import our two classes ---
from models.merton import MertonJumpDiffusion
from calibration.merton_calibrator import MertonCalibrator

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
print("All modules imported successfully.")

All modules imported successfully.


In [4]:
# Cell 2: Setup the Monte Carlo Test
# These are the "true" parameters we want to find
true_merton_params = {
    'S0': 100,
    'mu': 0.05,       # 5% total drift
    'sigma': 0.2,     # 20% diffusion vol
    'lambda_j': 5.0,  # 5 jumps per year
    'mu_j': -0.1,     # -10% mean jump
    'sigma_j': 0.05    # 5% jump vol
}

# --- Simulation setup ---
T = 10       # 1 year
dt = 1/252    # Daily steps
n_test_paths = 50 # How many tests to run. (100 is fast, 500 is more accurate)

# --- Result storage ---
calibrated_params_list = []

# --- Instantiate our models once ---
merton_model = MertonJumpDiffusion(**true_merton_params)
merton_cal = MertonCalibrator(dt=dt)

print(f"Running {n_test_paths} Merton calibration tests...")
print(f"True Params: {true_merton_params}")


# Cell 3: Run the Test Loop
# This cell will take a few minutes!
start_time = time.time()

# tqdm adds a progress bar
for _ in tqdm(range(n_test_paths)):
    # 1. Simulate one random path
    sim_path_array = merton_model.simulate(T, dt, n_paths=1).flatten()
    sim_series = pd.Series(sim_path_array)

    # 2. Calibrate this one path
    # We use a try/except, just in case one run fails
    try:
        params = merton_cal.fit(sim_series)
        calibrated_params_list.append(params)
    except Exception as e:
        # If one run fails, just append an empty dict
        calibrated_params_list.append({})

end_time = time.time()
print(f"--- Test Complete in {end_time - start_time:.2f} seconds ---")

# Cell 4: Analyze and Display Results
# 1. Convert our list of dicts into a pandas DataFrame
results_df = pd.DataFrame(calibrated_params_list)

# 2. Get the average of each parameter column
#    np.nanmean will safely ignore any failed runs
mean_mu = np.nanmean(results_df['mu'])
mean_sigma = np.nanmean(results_df['sigma'])
mean_lambda = np.nanmean(results_df['lambda_j'])
mean_mu_j = np.nanmean(results_df['mu_j'])
mean_sigma_j = np.nanmean(results_df['sigma_j'])

# 3. Print a clean comparison table
print("\n=== True vs Average Calibrated Parameters ===")
print(f"{'Parameter':<16}{'True Value':>15}{'Average Calibrated':>25}")
print("-" * 56)

print(f"{'μ (drift)':<16}{true_merton_params['mu']:>15.4f}{mean_mu:>25.4f}")
print(f"{'σ (diff vol)':<16}{true_merton_params['sigma']:>15.4f}{mean_sigma:>25.4f}")
print(f"{'λ (jump freq)':<16}{true_merton_params['lambda_j']:>15.4f}{mean_lambda:>25.4f}")
print(f"{'μ_j (jump mean)':<16}{true_merton_params['mu_j']:>15.4f}{mean_mu_j:>25.4f}")
print(f"{'σ_j (jump vol)':<16}{true_merton_params['sigma_j']:>15.4f}{mean_sigma_j:>25.4f}")

# 4. Check how many runs failed
failed_runs = results_df['mu'].isnull().sum()
print(f"\nTotal runs: {n_test_paths}")
print(f"Successful runs: {n_test_paths - failed_runs}")
print(f"Failed runs: {failed_runs}")

Running 50 Merton calibration tests...
True Params: {'S0': 100, 'mu': 0.05, 'sigma': 0.2, 'lambda_j': 5.0, 'mu_j': -0.1, 'sigma_j': 0.05}


100%|██████████| 50/50 [00:20<00:00,  2.43it/s]

--- Test Complete in 20.57 seconds ---

=== True vs Average Calibrated Parameters ===
Parameter            True Value       Average Calibrated
--------------------------------------------------------
μ (drift)                0.0500                   0.0630
σ (diff vol)             0.2000                   0.2000
λ (jump freq)            5.0000                   4.8546
μ_j (jump mean)         -0.1000                  -0.1024
σ_j (jump vol)           0.0500                   0.0468

Total runs: 50
Successful runs: 50
Failed runs: 0
